In [7]:
%load_ext autoreload
%autoreload 2

from scripts import process

import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.features import shapes, rasterize
from rasterio.mask import mask
from rasterio.enums import MergeAlg
from shapely.geometry import shape, Point, mapping, box
from tqdm import tqdm
import numpy as np
import folium
from folium import GeoJson, GeoJsonTooltip
from folium.plugins import Search
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
from branca.element import Template, MacroElement

from IPython.display import clear_output
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# === CONFIGURATION ===
raster_path = "data/gridded_pop/covariates/GRIDDED_POP.tif"
hospital_csv = "data/gridded_pop/hosmap.csv"
output_shp = "output/population_grid_epsg32647.shp"
pop_nhso_path = "data/gridded_pop/nhso_pop_2020.csv"
pop_dopa_path = "data/gridded_pop/dopa_pop_2020.csv"
value_column = "value"
target_crs = "EPSG:32647"

In [3]:
pop_nhso = pd.read_csv(pop_nhso_path)
pop_dopa, _, _ = process.DataCleaning.clean('dopa_pop_2020', 'data/gridded_pop', 'output', clear=True, export_df=False)
hos = pd.read_csv(hospital_csv)

pop_nhso['hcode'] = pop_nhso['hospital_code'].apply(lambda x: str(x[4:]))
pop_nhso = pop_nhso.merge(hos[['hcode', 'hname', 'prov', 'ampr', 'tumbon']], how='left', on='hcode')
pop_nhso = pop_nhso[~pop_nhso['tumbon'].isnull()]
pop_nhso['prov'] = pop_nhso['prov'].apply(lambda x: 'กรุงเทพฯ' if x in ['กรมแพทย์ทหารอากาศ', 'กรมแพทย์ทหารเรือ'] else x)

pop_subdist_uc = pop_nhso.groupby(['prov', 'ampr', 'tumbon'])['m_population'].sum().reset_index()

display(pop_subdist_uc)
display(pop_dopa)

File name                : dopa_pop_2020
Path to data dictionary  : data/gridded_pop/data_dict-dopa_pop_2020.xlsx
Path to cleaned dataset  : None


/var/folders/bf/rp_bmxh57x748ppg9b4stxy80000gn/T/ipykernel_1400/4288429274.py:3: DtypeWarning: Columns (13,16,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  hos = pd.read_csv(hospital_csv)


,prov,ampr,tumbon,m_population
0,กระบี่,คลองท่อม,คลองท่อมใต้,62464
1,กระบี่,ปลายพระยา,ปลายพระยา,29863
2,กระบี่,ลำทับ,ลำทับ,19528
3,กระบี่,อ่าวลึก,อ่าวลึกใต้,48324
4,กระบี่,เกาะลันตา,เกาะลันตาใหญ่,27766
...,...,...,...,...
1153,แม่ฮ่องสอน,ปาย,เวียงใต้,31686
1154,แม่ฮ่องสอน,สบเมย,แม่สวด,33139
1155,แม่ฮ่องสอน,เมืองแม่ฮ่องสอน,จองคำ,48404
1156,แม่ฮ่องสอน,แม่ลาน้อย,ขุนแม่ลาน้อย,27728


,month,prov_id,prov,dist_id,dist,subdist_id,subdist,village_id,village,pop_male,pop_female,pop_total,household
0,6312,0,ทั่วประเทศ,0,-,0,-,0,-,"32,375,532","33,811,195","66,186,727","27,224,743"
1,6312,10,กรุงเทพมหานคร,0,-,0,-,0,-,"2,625,938","2,962,284","5,588,222","3,103,483"
2,6312,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,0,-,0,-,"21,675","23,248","44,923","19,137"
3,6312,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,10010100,พระบรมมหาราชวัง,0,-,"1,977","1,369","3,346","1,207"
4,6312,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,10010200,วังบูรพาภิรมย์,0,-,"4,880","4,770","9,650","5,363"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11375,6312,96,จังหวัดนราธิวาส,9697,ท้องถิ่นเทศบาลเมืองตากใบ,96020100,ตำบลเจ๊ะเห,0,-,"9,557","10,129","19,686","5,659"
11376,6312,96,จังหวัดนราธิวาส,9698,ท้องถิ่นเทศบาลเมืองสุไหงโกลก,0,-,0,-,"19,312","21,510","40,822","15,395"
11377,6312,96,จังหวัดนราธิวาส,9698,ท้องถิ่นเทศบาลเมืองสุไหงโกลก,96100100,ตำบลสุไหงโก-ลก,0,-,"19,312","21,510","40,822","15,395"
11378,6312,96,จังหวัดนราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,0,-,0,-,"19,673","20,460","40,133","15,736"


In [4]:
# === Load hospitals and project ===
df_hos = pd.read_csv(hospital_csv)
gdf_hos = gpd.GeoDataFrame(df_hos, geometry=[Point(xy) for xy in zip(df_hos['longitude'], df_hos['latitude'])], crs="EPSG:4326")
gdf_hos = gdf_hos.to_crs(target_crs)

# === Filter only class P hospitals ===
gdf_hos['hclass'] = gdf_hos['hclass'].apply(lambda x: x.split(',') if pd.notnull(x) else [])
gdf_hos = gdf_hos[gdf_hos['hclass'].apply(lambda x: any(i in x for i in ['P', 'R0211', 'R0213', 'R0207']))].copy()

# === Identify hospitals with missing coordinates ===
null_coor = gdf_hos[gdf_hos['latitude'].isnull()]
coordinationmap_path = 'output/map/coordination.csv'

if null_coor.empty:
    print('No null coordinates.')
    
else:
    
    screen = input('Do you want to fill the null coordination for hospital? (Y/N): ').lower()

    if screen == 'y':
        
        print(f'{len(null_coor)} hospitals still need coordinates. Looking up from existing geocoded file...')
        
        # === Step 1: Try filling from CSV ===
        geocode_df = pd.read_csv(coordinationmap_path).dropna(subset=['latitude', 'longitude'])
        geocode_df['hcode'] = geocode_df['hcode'].astype(str).str.zfill(5)

        gdf_hos.set_index('hcode', inplace=True)
        geocode_df.set_index('hcode', inplace=True)       
       
        gdf_hos.update(geocode_df[['latitude', 'longitude']])
        gdf_hos.reset_index(inplace=True)
        
        # Reconstruct geometry
        gdf_hos['geometry'] = [Point(xy) if pd.notnull(xy[0]) and pd.notnull(xy[1]) else None for xy in zip(gdf_hos['longitude'], gdf_hos['latitude'])]
        gdf_hos.set_crs("EPSG:4326", inplace=True, allow_override=True)
        
        # === Step 2: Geocode remaining nulls via Google Maps API ===
        null_coor = gdf_hos[gdf_hos['latitude'].isnull()]  # Recheck

        if not null_coor.empty:
            print(f'{len(null_coor)} hospitals still need coordinates. Geocoding via Google Maps API...')
                    
            # === Setup Google Maps Geocoder ===
            with open('private/GoogleMap_API', 'r') as f:
                API_KEY = f.read().strip()
            geolocator = GoogleV3(api_key=API_KEY, timeout=10)
            geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.05)  # ~20 requests/sec
            
            updated_rows = []
            for idx, row in tqdm(null_coor.iterrows(), total=len(null_coor), desc="Geocoding hospitals"):
                address = f"{row['hname']}, {row['tumbon']}, {row['ampr']}, {row['prov']}"
                try:
                    location = geocode(address)
                    if location:
                        gdf_hos.at[idx, 'latitude'] = location.latitude
                        gdf_hos.at[idx, 'longitude'] = location.longitude
                        gdf_hos.at[idx, 'geometry'] = Point(location.longitude, location.latitude)
                        updated_rows.append({
                            'hcode': row['hcode'],
                            'hname': row['hname'],
                            'prov': row['prov'],
                            'ampr': row['ampr'],
                            'tumbon': row['tumbon'],
                            'latitude': location.latitude,
                            'longitude': location.longitude
                        })

                except Exception as e:
                    print(f"Error for {row['hcode']} - {address}: {e}")

                # Save results
                if len(updated_rows) % 100 == 0 and updated_rows:
                    pd.DataFrame(updated_rows).to_csv(coordinationmap_path, mode='a', header=False, index=False)
                    print(f"Saved {len(updated_rows)} newly geocoded records to {coordinationmap_path}")
                    updated_rows = []
            
            # Final save for remaining rows
            if updated_rows:
                pd.DataFrame(updated_rows).to_csv(coordinationmap_path, mode='a', header=False, index=False)
                print(f"Saved {len(updated_rows)} newly geocoded records to {coordinationmap_path}")

        else:
            print('All missing coordinates filled from file.')
    else:
        print("Skipping coordinate filling.")

/var/folders/bf/rp_bmxh57x748ppg9b4stxy80000gn/T/ipykernel_1400/917256489.py:2: DtypeWarning: Columns (13,16,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hos = pd.read_csv(hospital_csv)


4110 hospitals still need coordinates. Looking up from existing geocoded file...
All missing coordinates filled from file.


In [6]:
gdf_hos_cov = gdf_hos.copy()
gdf_hos_cov = gdf_hos_cov.dropna(subset=['latitude', 'longitude'])
gdf_hos_cov = gdf_hos_cov.to_crs(target_crs)

# === Create buffers ===
gdf_hos_cov['buffer_1km'] = gdf_hos_cov.geometry.buffer(1000)
gdf_hos_cov['buffer_5km'] = gdf_hos_cov.geometry.buffer(5000)
gdf_hos_cov['buffer_10km'] = gdf_hos_cov.geometry.buffer(10000)

# === Combine all buffers for union/intersection ===
buffer_1km_basic = gdf_hos_cov[gdf_hos_cov['hclass'].apply(lambda x: any(i in x for i in ['P']))]['buffer_1km'].union_all()
buffer_5km_basic = gdf_hos_cov[gdf_hos_cov['hclass'].apply(lambda x: any(i in x for i in ['P']))]['buffer_5km'].union_all()
buffer_10km_basic = gdf_hos_cov[gdf_hos_cov['hclass'].apply(lambda x: any(i in x for i in ['P']))]['buffer_10km'].union_all()
buffer_1km_inno = gdf_hos_cov[gdf_hos_cov['hclass'].apply(lambda x: any(i in x for i in ['R0211', 'R0213', 'R0207']))]['buffer_1km'].union_all()
buffer_5km_inno = gdf_hos_cov[gdf_hos_cov['hclass'].apply(lambda x: any(i in x for i in ['R0211', 'R0213', 'R0207']))]['buffer_5km'].union_all()
buffer_10km_inno = gdf_hos_cov[gdf_hos_cov['hclass'].apply(lambda x: any(i in x for i in ['R0211', 'R0213', 'R0207']))]['buffer_10km'].union_all()
buffer_1km_total = gdf_hos_cov['buffer_1km'].union_all()
buffer_5km_total = gdf_hos_cov['buffer_5km'].union_all()
buffer_10km_total = gdf_hos_cov['buffer_10km'].union_all()

# === Calculate population in each ring ===
def calculate_population(buffer_geom, raster_path, original_crs):
    try:
        with rasterio.open(raster_path) as src:
            raster_crs = src.crs
            
            # Reproject buffer to match raster CRS
            buffer_proj = gpd.GeoSeries([buffer_geom], crs=original_crs).to_crs(raster_crs).iloc[0]

            # Mask raster
            out = mask(
                src,
                [mapping(buffer_proj)],
                crop=True,
                nodata=src.nodata,
                all_touched=True
            )

            if not out or len(out) < 1:
                print("Empty output from mask (no overlap).")
                return 0

            out_image = out[0]  # safe now
            if out_image.size == 0:
                print("Masked array is empty.")
                return 0

            data = out_image[0].astype(float) if out_image.ndim == 3 else out_image.astype(float)

            if src.nodata is not None:
                data[data == src.nodata] = np.nan

            return np.nansum(data)
        
    except Exception as e:
        print(f"Skipping buffer due to error: {e}")
        return 0


original_crs = gdf_hos_cov.crs  # typically EPSG:32647 or similar

pop_1km_basic = calculate_population(buffer_1km_basic, raster_path, original_crs)
pop_5km_basic = calculate_population(buffer_5km_basic, raster_path, original_crs)
pop_10km_basic = calculate_population(buffer_10km_basic, raster_path, original_crs)
pop_1km_total = calculate_population(buffer_1km_total, raster_path, original_crs)
pop_5km_total = calculate_population(buffer_5km_total, raster_path, original_crs)
pop_10km_total = calculate_population(buffer_10km_total, raster_path, original_crs)

# === Print or save results ===
print("Coverage population before innovative facilities-----")
print(f"Population within 1 km: {pop_1km_basic:,.0f}")
print(f"Population within 5 km: {pop_5km_basic:,.0f}")
print(f"Population within 10 km: {pop_10km_basic:,.0f}")
print("Coverage population after innovative facilities-----")
print(f"Population within 1 km: {pop_1km_total:,.0f}")
print(f"Population within 5 km: {pop_5km_total:,.0f}")
print(f"Population within 10 km: {pop_10km_total:,.0f}")

results = []
with tqdm(total=len(gdf_hos_cov)) as pbar:
    for idx, row in gdf_hos_cov.iterrows():
        hname = row['hname']
        pbar.set_description(f"Hospital: {hname}")
        hclass = row['hclass']
        color = (
            'blue' if 'P' in hclass else
            'orange' if 'R0211' in hclass else
            'red' if 'R0213' in hclass else
            'green' if 'R0207' in hclass else
            'gray'
        )
        point_geom = row['geometry']
        buf1km = row['buffer_1km']
        buf5km = row['buffer_5km']
        buf10km = row['buffer_10km']
        pop_sum1km = calculate_population(buf1km, raster_path, original_crs)
        pop_sum5km = calculate_population(buf5km, raster_path, original_crs)
        pop_sum10km = calculate_population(buf10km, raster_path, original_crs)
        
        results.append({
            'hospital': hname,
            'hclass': hclass,
            'color': color,
            'pop_cover1km': pop_sum1km,
            'pop_cover5km': pop_sum5km,
            'pop_cover10km': pop_sum10km,
            'geometry': point_geom,
            'buf_1km': buf1km,
            'buf_5km': buf5km,
            'buf_10km': buf10km,
        })
        
        pbar.update(1)

clear_output()
gdf_buffer = pd.DataFrame(results)
gdf_buffer = gpd.GeoDataFrame(gdf_buffer, geometry='geometry', crs=target_crs)
display(gdf_buffer)

,hospital,hclass,color,pop_cover1km,pop_cover5km,pop_cover10km,geometry,buf_1km,buf_5km,buf_10km
0,รพ.สต.ทุ่งแค้ว,[P],blue,1400.733029,8527.596257,31106.718983,POINT (622495.78 2024894.717),"POLYGON ((623495.780233877 2024894.7168450116,...","POLYGON ((627495.780233877 2024894.7168450116,...","POLYGON ((632495.780233877 2024894.7168450116,..."
1,รพ.สต.บ่อ,[P],blue,143.878373,1710.898425,7938.541880,POINT (686021.19 2093337.353),"POLYGON ((687021.1896898694 2093337.352551678,...","POLYGON ((691021.1896898694 2093337.352551678,...","POLYGON ((696021.1896898694 2093337.352551678,..."
2,รพ.สต.ผาสิงห์,[P],blue,175.690091,5080.554420,41714.104490,POINT (684527.844 2083492.461),POLYGON ((685527.8444456792 2083492.4612469645...,POLYGON ((689527.8444456792 2083492.4612469645...,POLYGON ((694527.8444456792 2083492.4612469645...
3,รพ.สต.ไชยสถาน,[P],blue,555.330045,13891.522192,48985.107548,POINT (681285.287 2076564.398),POLYGON ((682285.2871571056 2076564.3975780327...,POLYGON ((686285.2871571056 2076564.3975780327...,POLYGON ((691285.2871571056 2076564.3975780327...
4,รพ.สต.ถืมตอง,[P],blue,889.602370,8994.582684,42893.102243,POINT (679426.661 2079070.361),POLYGON ((680426.6614737078 2079070.3605844479...,POLYGON ((684426.6614737078 2079070.3605844479...,POLYGON ((689426.6614737078 2079070.3605844479...
...,...,...,...,...,...,...,...,...,...,...
23849,ศักดิ์ศรีเภสัช สาขาประตูศึกษา มช.,[R0211],orange,6648.893658,98794.761799,261433.170209,POINT (495276.088 2077890.329),POLYGON ((496276.0875423514 2077890.3294405392...,POLYGON ((500276.0875423514 2077890.3294405392...,POLYGON ((505276.0875423514 2077890.3294405392...
23850,บ้านมุมยา,[R0211],orange,10354.218876,68680.079532,149966.254173,POINT (589810.575 1497005.851),POLYGON ((590810.5748744288 1497005.8511413455...,POLYGON ((594810.5748744288 1497005.8511413455...,POLYGON ((599810.5748744288 1497005.8511413455...
23851,มู่การยา,[R0211],orange,3717.967038,47688.477323,116875.868159,POINT (502938.915 2056620.076),POLYGON ((503938.91524732154 2056620.076238037...,POLYGON ((507938.91524732154 2056620.076238037...,POLYGON ((512938.91524732154 2056620.076238037...
23852,ร้านศักดิ์ศรีเภสัช,[R0211],orange,5686.989081,88128.988352,250964.488440,POINT (495294.298 2079612.4),"POLYGON ((496294.2976515578 2079612.399643632,...","POLYGON ((500294.2976515578 2079612.399643632,...","POLYGON ((505294.2976515578 2079612.399643632,..."


In [26]:
# Create a dictionary to store output arrays
buffer_arrays = {}

# Load hospital data
gdf_hos_den = gdf_hos.copy()
gdf_hos_den = gdf_hos_den.dropna(subset=['latitude', 'longitude'])
gdf_hos_den = gdf_hos_den.to_crs(target_crs)
gdf_hos_den['buffer_1km'] = gdf_hos_den.geometry.buffer(1000)
gdf_hos_den['buffer_5km'] = gdf_hos_den.geometry.buffer(5000)
gdf_hos_den['buffer_10km'] = gdf_hos_den.geometry.buffer(10000)

# Base raster information
base_grid_path = 'data/gridded_pop/covariates/BLANK_100m.tif'
raster_path = 'data/gridded_pop/covariates/GRIDDED_POP.tif'
buffer_cols = ['buffer_1km', 'buffer_5km', 'buffer_10km']

with rasterio.open(base_grid_path) as base:
    base_shape = (base.height, base.width)
    base_transform = base.transform
    base_crs = base.crs

# Count overlapping buffer zones per raster cell
for buffer in buffer_cols:
    shapes = [(geom, 1) for geom in gdf_hos_den[gdf_hos_den['hclass'].apply(lambda x: any(i in x for i in ['P']))][buffer]]
    array = rasterize(
        shapes,
        out_shape=base_shape,
        transform=base_transform,
        fill=1,
        merge_alg=MergeAlg.add,
        dtype=np.float32
    )
    buffer_arrays[f'{buffer}_basic'] = array
    
    shapes = [(geom, 1) for geom in gdf_hos_den[buffer]]
    array = rasterize(
        shapes,
        out_shape=base_shape,
        transform=base_transform,
        fill=1,
        merge_alg=MergeAlg.add,
        dtype=np.float32
    )
    buffer_arrays[f'{buffer}_inno'] = array

clear_output()

# Read original population raster
with rasterio.open(raster_path) as src:
    buffer_arrays['pop_total'] = src.read(1).astype(np.float32)
    pop_meta = src.meta.copy()

# Adjusted population: divide by buffer count, avoid divide-by-zero
for km in ['1km_basic', '1km_inno', '5km_basic', '5km_inno', '10km_basic', '10km_inno']:
    buffer = f'buffer_{km}'
    adjusted = buffer_arrays['pop_total'] / buffer_arrays[buffer]
    buffer_arrays[f'pop_adj_{km}'] = np.nan_to_num(adjusted, nan=0.0)
    
for km in ['1km_basic', '1km_inno', '5km_basic', '5km_inno', '10km_basic', '10km_inno']:    
    buffer = f'buffer_{km}'
    array = buffer_arrays[buffer]  # assuming array already exists here
    if np.unique(array).size > 1:
        print(f"{buffer} has more than one unique value.")
    else:
        print(f"{buffer} has only one unique value.")

# Save adjusted population rasters
for km in ['1km_basic', '1km_inno', '5km_basic', '5km_inno', '10km_basic', '10km_inno']:
    out_path = f'data/gridded_pop/covariates/GRIDDED_POP_ADJ_{km}.tif'
    out_array = buffer_arrays[f'pop_adj_{km}']

    with rasterio.open(
        out_path, 'w',
        driver='GTiff',
        height=base_shape[0],
        width=base_shape[1],
        count=1,
        dtype='float32',
        crs=base_crs,
        transform=base_transform
    ) as dst:
        dst.write(out_array, 1)

raster_path_1km_basic = 'data/gridded_pop/covariates/GRIDDED_POP_ADJ_1km_basic.tif'
raster_path_5km_basic = 'data/gridded_pop/covariates/GRIDDED_POP_ADJ_5km_basic.tif'
raster_path_10km_basic = 'data/gridded_pop/covariates/GRIDDED_POP_ADJ_10km_basic.tif'
raster_path_1km_inno = 'data/gridded_pop/covariates/GRIDDED_POP_ADJ_1km_inno.tif'
raster_path_5km_inno = 'data/gridded_pop/covariates/GRIDDED_POP_ADJ_5km_inno.tif'
raster_path_10km_inno = 'data/gridded_pop/covariates/GRIDDED_POP_ADJ_10km_inno.tif'

results = []
with tqdm(total=len(gdf_hos_cov)) as pbar:
    for idx, row in gdf_hos_cov.iterrows():
        hname = row['hname']
        pbar.set_description(f"Hospital: {hname}")
        hclass = row['hclass']
        color = (
            'blue' if 'P' in hclass else
            'orange' if 'R0211' in hclass else
            'red' if 'R0213' in hclass else
            'green' if 'R0207' in hclass else
            'gray'
        )
        point_geom = row['geometry']
        buf1km = row['buffer_1km']
        buf5km = row['buffer_5km']
        buf10km = row['buffer_10km']
        pop_sum1km_basic = calculate_population(buf1km, raster_path_1km_basic, original_crs)
        pop_sum5km_basic = calculate_population(buf5km, raster_path_5km_basic, original_crs)
        pop_sum10km_basic = calculate_population(buf10km, raster_path_10km_basic, original_crs)
        pop_sum1km_inno = calculate_population(buf1km, raster_path_1km_inno, original_crs)
        pop_sum5km_inno = calculate_population(buf5km, raster_path_5km_inno, original_crs)
        pop_sum10km_inno = calculate_population(buf10km, raster_path_10km_inno, original_crs)
        
        results.append({
            'hospital': hname,
            'hclass': hclass,
            'color': color,
            'pop_cover1km_basic': pop_sum1km_basic,
            'pop_cover5km_basic': pop_sum5km_basic,
            'pop_cover10km_basic': pop_sum10km_basic,
            'pop_cover1km_inno': pop_sum1km_inno,
            'pop_cover5km_inno': pop_sum5km_inno,
            'pop_cover10km_inno': pop_sum10km_inno,
            'geometry': point_geom,
            'buf_1km': buf1km,
            'buf_5km': buf5km,
            'buf_10km': buf10km,
        })
        
        pbar.update(1)

clear_output()
gdf_buffer_adj = pd.DataFrame(results)
gdf_buffer_adj = gpd.GeoDataFrame(gdf_buffer_adj, geometry='geometry', crs=target_crs)
display(gdf_buffer_adj)


,hospital,hclass,color,pop_cover1km_basic,pop_cover5km_basic,pop_cover10km_basic,pop_cover1km_inno,pop_cover5km_inno,pop_cover10km_inno,geometry,buf_1km,buf_5km,buf_10km
0,รพ.สต.ทุ่งแค้ว,[P],blue,732.389133,1285.409492,1818.535752,732.269257,1012.506434,1273.684007,POINT (622495.78 2024894.717),"POLYGON ((623495.780233877 2024894.7168450116,...","POLYGON ((627495.780233877 2024894.7168450116,...","POLYGON ((632495.780233877 2024894.7168450116,..."
1,รพ.สต.บ่อ,[P],blue,75.291753,828.244038,1385.141788,75.291753,828.244038,1223.054017,POINT (686021.19 2093337.353),"POLYGON ((687021.1896898694 2093337.352551678,...","POLYGON ((691021.1896898694 2093337.352551678,...","POLYGON ((696021.1896898694 2093337.352551678,..."
2,รพ.สต.ผาสิงห์,[P],blue,95.461664,1151.743519,2732.413783,95.461664,774.146440,1608.933110,POINT (684527.844 2083492.461),POLYGON ((685527.8444456792 2083492.4612469645...,POLYGON ((689527.8444456792 2083492.4612469645...,POLYGON ((694527.8444456792 2083492.4612469645...
3,รพ.สต.ไชยสถาน,[P],blue,309.786451,1870.634637,2670.853221,235.146949,875.104918,1422.017733,POINT (681285.287 2076564.398),POLYGON ((682285.2871571056 2076564.3975780327...,POLYGON ((686285.2871571056 2076564.3975780327...,POLYGON ((691285.2871571056 2076564.3975780327...
4,รพ.สต.ถืมตอง,[P],blue,452.116776,1590.523175,2465.115443,437.904752,915.159583,1337.463283,POINT (679426.661 2079070.361),POLYGON ((680426.6614737078 2079070.3605844479...,POLYGON ((684426.6614737078 2079070.3605844479...,POLYGON ((689426.6614737078 2079070.3605844479...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23849,ศักดิ์ศรีเภสัช สาขาประตูศึกษา มช.,[R0211],orange,6383.276616,3669.539093,4816.074966,1118.790637,943.795274,1411.409327,POINT (495276.088 2077890.329),POLYGON ((496276.0875423514 2077890.3294405392...,POLYGON ((500276.0875423514 2077890.3294405392...,POLYGON ((505276.0875423514 2077890.3294405392...
23850,บ้านมุมยา,[R0211],orange,4130.835310,5526.680689,4843.110781,1550.333303,2215.357726,2712.358347,POINT (589810.575 1497005.851),POLYGON ((590810.5748744288 1497005.8511413455...,POLYGON ((594810.5748744288 1497005.8511413455...,POLYGON ((599810.5748744288 1497005.8511413455...
23851,มู่การยา,[R0211],orange,3382.310502,7600.862178,5468.536284,671.449764,1348.745320,1481.196011,POINT (502938.915 2056620.076),POLYGON ((503938.91524732154 2056620.076238037...,POLYGON ((507938.91524732154 2056620.076238037...,POLYGON ((512938.91524732154 2056620.076238037...
23852,ร้านศักดิ์ศรีเภสัช,[R0211],orange,2890.443198,3431.599094,4653.161319,863.115245,904.297017,1353.414117,POINT (495294.298 2079612.4),"POLYGON ((496294.2976515578 2079612.399643632,...","POLYGON ((500294.2976515578 2079612.399643632,...","POLYGON ((505294.2976515578 2079612.399643632,..."


In [27]:
gdf_buffer_merge = gdf_buffer.merge(
    gdf_buffer_adj,
    left_index=True,
    right_index=True,
    how='left',
    suffixes=['', 'adj']
)

dens_before = gdf_buffer_merge['pop_cover1km_basic'].mean()
dens_after = gdf_buffer_merge['pop_cover1km_inno'].mean()

print(f'Density (Before) : {dens_before}')
print(f'Density (After) : {dens_after}')
display(gdf_buffer_merge)

Density (Before) : 1729.900931919569
Density (After) : 706.3710840254499


,hospital,hclass,color,pop_cover1km,pop_cover5km,pop_cover10km,geometry,buf_1km,buf_5km,buf_10km,hospitaladj,hclassadj,coloradj,pop_cover1km_basic,pop_cover5km_basic,pop_cover10km_basic,pop_cover1km_inno,pop_cover5km_inno,pop_cover10km_inno,geometryadj,buf_1kmadj,buf_5kmadj,buf_10kmadj
0,รพ.สต.ทุ่งแค้ว,[P],blue,1400.733029,8527.596257,31106.718983,POINT (622495.78 2024894.717),"POLYGON ((623495.780233877 2024894.7168450116,...","POLYGON ((627495.780233877 2024894.7168450116,...","POLYGON ((632495.780233877 2024894.7168450116,...",รพ.สต.ทุ่งแค้ว,[P],blue,732.389133,1285.409492,1818.535752,732.269257,1012.506434,1273.684007,POINT (622495.78 2024894.717),"POLYGON ((623495.780233877 2024894.7168450116,...","POLYGON ((627495.780233877 2024894.7168450116,...","POLYGON ((632495.780233877 2024894.7168450116,..."
1,รพ.สต.บ่อ,[P],blue,143.878373,1710.898425,7938.541880,POINT (686021.19 2093337.353),"POLYGON ((687021.1896898694 2093337.352551678,...","POLYGON ((691021.1896898694 2093337.352551678,...","POLYGON ((696021.1896898694 2093337.352551678,...",รพ.สต.บ่อ,[P],blue,75.291753,828.244038,1385.141788,75.291753,828.244038,1223.054017,POINT (686021.19 2093337.353),"POLYGON ((687021.1896898694 2093337.352551678,...","POLYGON ((691021.1896898694 2093337.352551678,...","POLYGON ((696021.1896898694 2093337.352551678,..."
2,รพ.สต.ผาสิงห์,[P],blue,175.690091,5080.554420,41714.104490,POINT (684527.844 2083492.461),POLYGON ((685527.8444456792 2083492.4612469645...,POLYGON ((689527.8444456792 2083492.4612469645...,POLYGON ((694527.8444456792 2083492.4612469645...,รพ.สต.ผาสิงห์,[P],blue,95.461664,1151.743519,2732.413783,95.461664,774.146440,1608.933110,POINT (684527.844 2083492.461),POLYGON ((685527.8444456792 2083492.4612469645...,POLYGON ((689527.8444456792 2083492.4612469645...,POLYGON ((694527.8444456792 2083492.4612469645...
3,รพ.สต.ไชยสถาน,[P],blue,555.330045,13891.522192,48985.107548,POINT (681285.287 2076564.398),POLYGON ((682285.2871571056 2076564.3975780327...,POLYGON ((686285.2871571056 2076564.3975780327...,POLYGON ((691285.2871571056 2076564.3975780327...,รพ.สต.ไชยสถาน,[P],blue,309.786451,1870.634637,2670.853221,235.146949,875.104918,1422.017733,POINT (681285.287 2076564.398),POLYGON ((682285.2871571056 2076564.3975780327...,POLYGON ((686285.2871571056 2076564.3975780327...,POLYGON ((691285.2871571056 2076564.3975780327...
4,รพ.สต.ถืมตอง,[P],blue,889.602370,8994.582684,42893.102243,POINT (679426.661 2079070.361),POLYGON ((680426.6614737078 2079070.3605844479...,POLYGON ((684426.6614737078 2079070.3605844479...,POLYGON ((689426.6614737078 2079070.3605844479...,รพ.สต.ถืมตอง,[P],blue,452.116776,1590.523175,2465.115443,437.904752,915.159583,1337.463283,POINT (679426.661 2079070.361),POLYGON ((680426.6614737078 2079070.3605844479...,POLYGON ((684426.6614737078 2079070.3605844479...,POLYGON ((689426.6614737078 2079070.3605844479...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23849,ศักดิ์ศรีเภสัช สาขาประตูศึกษา มช.,[R0211],orange,6648.893658,98794.761799,261433.170209,POINT (495276.088 2077890.329),POLYGON ((496276.0875423514 2077890.3294405392...,POLYGON ((500276.0875423514 2077890.3294405392...,POLYGON ((505276.0875423514 2077890.3294405392...,ศักดิ์ศรีเภสัช สาขาประตูศึกษา มช.,[R0211],orange,6383.276616,3669.539093,4816.074966,1118.790637,943.795274,1411.409327,POINT (495276.088 2077890.329),POLYGON ((496276.0875423514 2077890.3294405392...,POLYGON ((500276.0875423514 2077890.3294405392...,POLYGON ((505276.0875423514 2077890.3294405392...
23850,บ้านมุมยา,[R0211],orange,10354.218876,68680.079532,149966.254173,POINT (589810.575 1497005.851),POLYGON ((590810.5748744288 1497005.8511413455...,POLYGON ((594810.5748744288 1497005.8511413455...,POLYGON ((599810.5748744288 1497005.8511413455...,บ้านมุมยา,[R0211],orange,4130.835310,5526.680689,4843.110781,1550.333303,2215.357726,2712.358347,POINT (589810.575 1497005.851),POLYGON ((590810.5748744288 1497005.8511413455...,POLYGON ((594810.5748744288

In [29]:
gdf_buffer_merge.to_csv('output/analyse/changeDensity.csv')

In [28]:
# === STEP 4: Plot ===
thailand = gpd.read_file('data/gridded_pop/tha_adm/tha_admbnda_adm0_rtsd_20220121.shp').to_crs(epsg=4326)
provinces = gpd.read_file('data/gridded_pop/tha_adm/tha_admbnda_adm1_rtsd_20220121.shp').to_crs(epsg=4326)
districts = gpd.read_file('data/gridded_pop/tha_adm/tha_admbnda_adm2_rtsd_20220121.shp').to_crs(epsg=4326)
provinces['name'] = provinces['ADM1_TH']
districts['name'] = districts.apply(lambda row: f'{row['ADM2_TH']}, {row['ADM1_TH']}', axis=1)

thailand = thailand[['geometry']]
provinces = provinces[['name', 'geometry']]
districts = districts[['name', 'geometry']]

# Reproject ring geometries to EPSG:4326
buffer_1km_basic_4326 = gpd.GeoSeries([buffer_1km_basic], crs=target_crs).to_crs(epsg=4326)
buffer_5km_basic_4326 = gpd.GeoSeries([buffer_5km_basic], crs=target_crs).to_crs(epsg=4326)
buffer_10km_basic_4326 = gpd.GeoSeries([buffer_10km_basic], crs=target_crs).to_crs(epsg=4326)
buffer_1km_inno_4326 = gpd.GeoSeries([buffer_1km_inno], crs=target_crs).to_crs(epsg=4326)
buffer_5km_inno_4326 = gpd.GeoSeries([buffer_5km_inno], crs=target_crs).to_crs(epsg=4326)
buffer_10km_inno_4326 = gpd.GeoSeries([buffer_10km_inno], crs=target_crs).to_crs(epsg=4326)
gdf_buffer_4326 = gdf_buffer_merge.to_crs(epsg=4326)

# === Step 1: Base map ===
rep_point = thailand.geometry.representative_point().iloc[0]
m = folium.Map(location=[rep_point.y, rep_point.x], zoom_start=6, tiles='cartodbpositron', control_scale=True)

# === Step 2: Provinces and search ===
prov = folium.GeoJson(
    provinces,
    name="Provinces",
    tooltip=folium.GeoJsonTooltip(fields=['name']),
    style_function=lambda x: {'fillOpacity': 0, 'color': 'gray', 'weight': 0}
).add_to(m)

dist = folium.GeoJson(
    districts,
    name="Districts",
    tooltip=folium.GeoJsonTooltip(fields=['name']),
    style_function=lambda x: {'fillOpacity': 0, 'color': 'gray', 'weight': 0}
).add_to(m)

Search(
    layer=prov,
    geom_type='Polygon',
    search_label='name',
    placeholder='Search province...',
    collapsed=False,
    position='topleft'
).add_to(m)

Search(
    layer=dist,
    geom_type='Polygon',
    search_label='name',
    placeholder='Search district...',
    collapsed=False,
    position='topleft'
).add_to(m)

# Buffer 10 km
buffer_10km_basic_layer = folium.FeatureGroup(name='Buffer 10 km (Basic Facilities)', show=False)
folium.GeoJson(
    buffer_10km_basic_4326.geometry[0],
    style_function=lambda x: {'fillColor': 'yellow', 'color': 'yellow', 'weight': 1, 'fillOpacity': 0.2, 'interactive': False}
).add_to(buffer_10km_basic_layer)
buffer_10km_basic_layer.add_to(m)

buffer_10km_inno_layer = folium.FeatureGroup(name='Buffer 10 km (Innovative Facilities)', show=False)
folium.GeoJson(
    buffer_10km_inno_4326.geometry[0],
    style_function=lambda x: {'fillColor': 'orange', 'color': 'orange', 'weight': 1, 'fillOpacity': 0.3, 'interactive': False}
).add_to(buffer_10km_inno_layer)
buffer_10km_inno_layer.add_to(m)

# Buffer 5 km
buffer_5km_basic_layer = folium.FeatureGroup(name='Buffer 5 km (Basic Facilities)', show=False)
folium.GeoJson(
    buffer_5km_basic_4326.geometry[0],
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 1, 'fillOpacity': 0.4, 'interactive': False}
).add_to(buffer_5km_basic_layer)
buffer_5km_basic_layer.add_to(m)

buffer_5km_inno_layer = folium.FeatureGroup(name='Buffer 5 km (Innovative Facilities)', show=False)
folium.GeoJson(
    buffer_5km_inno_4326.geometry[0],
    style_function=lambda x: {'fillColor': 'purple', 'color': 'purple', 'weight': 1, 'fillOpacity': 0.5, 'interactive': False}
).add_to(buffer_5km_inno_layer)
buffer_5km_inno_layer.add_to(m)

# Buffer 1 km
buffer_1km_basic_layer = folium.FeatureGroup(name='Buffer 1 km (Basic Facilities)', show=False)
folium.GeoJson(
    buffer_1km_basic_4326.geometry[0],
    style_function=lambda x: {'fillColor': 'pink', 'color': 'pink', 'weight': 1, 'fillOpacity': 0.6, 'interactive': False}
).add_to(buffer_1km_basic_layer)
buffer_1km_basic_layer.add_to(m)

buffer_1km_inno_layer = folium.FeatureGroup(name='Buffer 1 km (Innovative Facilities)', show=False)
folium.GeoJson(
    buffer_1km_inno_4326.geometry[0],
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.6, 'interactive': False}
).add_to(buffer_1km_inno_layer)
buffer_1km_inno_layer.add_to(m)

hospital_layer = folium.FeatureGroup(name='Hospitals', show=True)
for _, row in tqdm(gdf_buffer_4326.iterrows(), total=len(gdf_buffer_4326), desc="Placing hospitals"):
    popup_text = f"""<b>{row['hospital']}</b><br>
                     <b>Crude coverage</b><br>
                     Pop. within 1 km: {row['pop_cover1km']:,.0f}<br>
                     Pop. within 5 km: {row['pop_cover5km']:,.0f}<br>
                     Pop. within 10 km: {row['pop_cover10km']:,.0f}<br><br>
                     <b>Adjusted coverage</b><br>
                     <b>Before Innovative Facilities</b><br>
                     Pop. within 1 km: {row['pop_cover1km_basic']:,.0f}<br>
                     Pop. within 5 km: {row['pop_cover5km_basic']:,.0f}<br>
                     Pop. within 10 km: {row['pop_cover10km_basic']:,.0f}<br>                     
                     <b>After Innovative Facilities</b><br>
                     Pop. within 1 km: {row['pop_cover1km_inno']:,.0f}<br>
                     Pop. within 5 km: {row['pop_cover5km_inno']:,.0f}<br>
                     Pop. within 10 km: {row['pop_cover10km_inno']:,.0f}"""
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color=row['color'],
        fill=True,
        fill_opacity=0.9,
        tooltip=popup_text
    ).add_to(hospital_layer)
hospital_layer.add_to(m)

# === STEP 7: Add population coverage summary box ===
html = f"""
    {{% macro html(this, kwargs) %}}
    <div style="
        position: fixed; 
        bottom: 50px; left: 50px; width: 320px; 
        z-index: 9999; 
        background-color: white; 
        padding: 10px; 
        border: 2px solid gray; 
        border-radius: 10px;
        box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
        font-size: 12px;
    ">
    <h4 style="margin: 0 0 5px 0;">Coverage Population Summary</h4>
    <b>Before Innovative Facilities</b><br>
    1 km: {pop_1km_basic:,.0f}<br>
    5 km: {pop_5km_basic:,.0f}<br>
    10 km: {pop_10km_basic:,.0f}<br>
    Density per facility in 1 km: {dens_before:,.0f}<br><br>
    <b>After Innovative Facilities</b><br>
    1 km: {pop_1km_total:,.0f}<br>
    5 km: {pop_5km_total:,.0f}<br>
    10 km: {pop_10km_total:,.0f}<br>
    Density per facility in 1 km: {dens_after:,.0f}<br><br>
    </div>
    {{% endmacro %}}
    """
popup_box = MacroElement()
popup_box._template = Template(html)
m.get_root().add_child(popup_box)

# === STEP 8: Add layer control and display map ===
folium.LayerControl(collapsed=False).add_to(m)
print('Finised Builting the Maps')
m.save('spatial_cov.html')

Placing hospitals: 100%|██████████| 23854/23854 [00:01<00:00, 22440.01it/s]


Finised Builting the Maps
